In [1]:
import lib.io.stan

csv_fname = 'results/exp10/exp10.7/samples_sigma0.10000_eps0.10000.csv'
nwarmup = 200
nsampling = 200
variables_of_interest = ['lp__','accept_stat__','stepsize__','treedepth__','n_leapfrog__',\
                         'divergent__', 'energy__','x0',  'x', 'mu_seeg', 'amplitude', 'offset',\
                         'x_init', 'z_init', 'epsilon']
pstr_samples_1 = lib.io.stan.read_samples(csv_fname,nwarmup,nsampling) # read sampler diagnostics and x0 for all sampling iterations

In [ ]:
%matplotlib inline
import lib.plots.stan
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(20,10))
plt.subplot(211)
plt.violinplot(pstr_samples_1['x0'][150:]);
plt.axhline(-2.0,color='red')
xtick_labels = []
for i in range(84):
    if(i%2 == 0):
        xtick_labels.append(str(i+1))
    else:
        xtick_labels.append('')
plt.xticks(np.r_[1:85],xtick_labels);
plt.xlabel('Region#',fontsize=15);
plt.ylabel('$x_0$',fontsize=15);

import lib.plots.stan

lib.plots.stan.nuts_diagnostics(pstr_samples_1)

In [ ]:
ez = [59]
pz = [74, 82]
plt.figure(figsize=(15,10))
ax_src_x = plt.subplot(211)
ax_src_z = plt.subplot(212)
sample_idx = -1
for i in range(84):
    if i in ez: 
        ax_src_x.plot(pstr_samples_1['x'][sample_idx,:,i], color='red');
        ax_src_z.plot(pstr_samples_1['z'][sample_idx,:,i], color='red');
    elif i in pz:
        ax_src_x.plot(pstr_samples_1['x'][sample_idx,:,i], color='orange');
        ax_src_z.plot(pstr_samples_1['z'][sample_idx,:,i], color='orange');
    else:
        ax_src_x.plot(pstr_samples_1['x'][sample_idx,:,i], color='black', alpha=0.1);
        ax_src_z.plot(pstr_samples_1['z'][sample_idx,:,i], color='black', alpha=0.1);
ax_src_x.set_title("Source activity - x")
ax_src_z.set_title("Source activity - z")

Phase space plots

In [ ]:
for i in range(84):
    if i in ez: 
        plt.plot(pstr_samples_1['x'][-1,:,i], pstr_samples_1['z'][-1,:,i], color='red')
    elif i in pz:
        plt.plot(pstr_samples_1['x'][-1,:,i], pstr_samples_1['z'][-1,:,i], color='orange')
    else:
        plt.plot(pstr_samples_1['x'][-1,:,i], pstr_samples_1['z'][-1,:,i], color='black')

Downsampling **seeg log power**

In [ ]:
%reset -f
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
fit_trgt = np.load('datasets/id001_ac/AC_fit_trgt.npz')['fit_trgt']
fit_trgt_ds = fit_trgt[0:-1:20,:]
plt.figure(figsize=(15,5))
plt.plot(fit_trgt,alpha=0.3);
plt.xlabel('time')
plt.ylabel('slp')
plt.title('Original')
plt.figure(figsize=(15,5))
plt.plot(fit_trgt_ds,alpha=0.3);
plt.xlabel('time')
plt.ylabel('slp')
plt.title('Downsampled')